# Dutch UMLS to concept table
This notebook describes how to create a UMLS concept table containing Dutch names, to be used in a named entity recognition and linking tool such as MedCAT. In the second part of this notebook, names from Dutch SNOMED are added. In the third part, English drug names are added, because these concepts are not well represented in the Dutch UMLS source vocabularies. 

Mapping from SNOMED Dutch to UMLS can be difficult because of many-to-many mapping, explained in this notebook.

Requirements:
- UMLS MySQL database containing Dutch ontologies.

For adding Dutch SNOMED names:
- Dutch SNOMED concept table, created in `dutch-snomed_to_concept-table.ipynb`
- UMLS MySQL database containing SNOMED-US, which is used for mapping SNOMED Dutch -> UMLS

For adding English drug names:
- UMLS MySQL database containing English drug ontologies such as RXNORM, ATC and Drugbank

In [1]:
import json
import os
import re
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display
from pathlib import Path
from sqlalchemy import create_engine
from utils import clean_name_status_column, convert_title_to_lowercase

pd.set_option('max_colwidth', 400)

# Set output version of the generated UMLS dutch concept table
UMLS_DUTCH_VERSION = 'v1.11'

# Set version of SNOMED to append to UMLS terms
snomed_dutch_file = Path('04_ConceptDB') / 'snomedct-dutch_v1.3.csv'

# Set custom concepts file
custom_concepts_file = Path("05_CustomConcepts") / "dutch-umls_custom_concepts.csv"

# Output files
output_file = Path("04_ConceptDB") / f'umls-dutch_{UMLS_DUTCH_VERSION}.csv'
output_file_with_drug_names = Path('04_ConceptDB') / f'umls-dutch_{UMLS_DUTCH_VERSION}_with_drugs_filtered-categories.csv'

In [2]:
# Credentials to connect to UMLS MySQL database
load_dotenv()
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
database = os.getenv('MYSQL_DATABASE')

# Create the connection
connection_string = f'mysql://{user}:{password}@{host}:{port}/{database}'
connection = create_engine(connection_string)

## All languages

In [5]:
# Retrieve Dutch UMLS concepts
query = """
SELECT cui, str, tty, sab FROM MRCONSO
"""
umls_original = pd.read_sql_query(query, con=connection)
umls_original

,cui,str,tty,sab
0,C0000039,"1,2-dipalmitoylphosphatidylcholine",IN,RXNORM
1,C0000039,"1,2-dipalmitoylphosphatidylcholine",PN,MTH
2,C0000052,"1,4-alpha-Glucan Branching Enzyme",PN,MTH
3,C0000052,"1,4-alpha-Glucan branching enzyme",PT,SNOMEDCT_US
4,C0000052,Branching enzyme,SY,SNOMEDCT_US
...,...,...,...,...
1757108,C5699990,MAKOplasty of hip joint,PT,SNOMEDCT_US
1757109,C5699990,MAKOplasty of hip joint (procedure),FN,SNOMEDCT_US
1757110,C5699991,SNOMED Clinical Terms version: 20220630 [R] (June 2022 Release),SY,SNOMEDCT_US
1757111,C5699992,SNOMEDCT_US_2022_09_01 to ICD10CM_2023 Mappings,XM,SNOMEDCT_US


## Retrieve medical concepts

In [6]:
# Retrieve Dutch UMLS concepts
query = """
SELECT cui, str, tty, sab FROM MRCONSO WHERE LAT = 'DUT'
"""
dutch_umls_original = pd.read_sql_query(query, con=connection)
dutch_umls_original.head()

,cui,str,tty,sab
0,C0000696,A-zenuwvezels,MH,MSHDUT
1,C0000715,Abattoir,MH,MSHDUT
2,C0000715,Abattoirs,SY,MSHDUT
3,C0000722,Abbreviated Injury Scale,MH,MSHDUT
4,C0000726,Buik,MH,MSHDUT


In [7]:
dutch_umls_original

,cui,str,tty,sab
0,C0000696,A-zenuwvezels,MH,MSHDUT
1,C0000715,Abattoir,MH,MSHDUT
2,C0000715,Abattoirs,SY,MSHDUT
3,C0000722,Abbreviated Injury Scale,MH,MSHDUT
4,C0000726,Buik,MH,MSHDUT
...,...,...,...,...
290051,C5690008,samenvattende ontslagbrief:bevinding:moment:{instelling}:document:podologie,LN,LNC-NL-NL
290052,C5690009,samenvattende ontslagbrief:bevinding:moment:{instelling}:document:chiropractie,LN,LNC-NL-NL
290053,C5690010,samenvattende ontslagbrief:bevinding:moment:{instelling}:document:cardiochirurgie,LN,LNC-NL-NL
290054,C5690011,samenvattende ontslagbrief:bevinding:moment:{instelling}:document:borstvoeding,LN,LNC-NL-NL


In [8]:
dutch_umls_original['sab'].unique()
df = dutch_umls_original

In [9]:
df[df['cui'] == 'C0037747']

,cui,str,tty,sab
41311,C0037747,Spanje,MH,MSHDUT


## Manual corrections
Some manual corrections. Easiest to do this as close to the source as possible, so they are processed downstream correctly.

In [10]:
# Correct Respiratory Failure
# Respiratory Insufficiency / C0035229 / 409623005 / https://uts.nlm.nih.gov/uts/umls/concept/C0035229
# Respiratory Failure / C1145670 / 409622000 / https://uts.nlm.nih.gov/uts/umls/concept/C1145670
display(dutch_umls_original.loc[dutch_umls_original.cui == 'C0035229'])
display(dutch_umls_original.loc[dutch_umls_original.cui == 'C1145670'])

,cui,str,tty,sab
38371,C0035229,"Insufficiëntie, respiratoire",MH,MSHDUT
38372,C0035229,Respiratoire insufficiëntie,SY,MSHDUT
38373,C0035229,insufficiëntie; ademhaling,PT,ICPC2ICD10DUT
38374,C0035229,insufficiëntie; respiratoir,PT,ICPC2ICD10DUT
38375,C0035229,respiratoir; insufficiëntie,PT,ICPC2ICD10DUT
38376,C0035229,longinsufficiëntie,LLT,MDRDUT


,cui,str,tty,sab
175955,C1145670,Ademhalingsinsufficiëntie,MH,MSHDUT
175956,C1145670,"Respiratoire insufficiëntie, niet gespecificeerd",PT,ICD10DUT
175957,C1145670,respiratoir falen,LLT,MDRDUT
175958,C1145670,respiratoir falen,LLT,MDRDUT
175959,C1145670,respiratoir falen,LLT,MDRDUT
175960,C1145670,respiratoir falen,PT,MDRDUT


In [11]:
dutch_umls_original.loc[dutch_umls_original.str == 'Respiratoire insufficiëntie, niet gespecificeerd', 'cui'] = 'C0035229'
dutch_umls_original.loc[dutch_umls_original.str == 'Ademhalingsinsufficiëntie', 'cui'] = 'C0035229'
display(dutch_umls_original.loc[dutch_umls_original.cui == 'C0035229'])
display(dutch_umls_original.loc[dutch_umls_original.cui == 'C1145670'])

,cui,str,tty,sab
38371,C0035229,"Insufficiëntie, respiratoire",MH,MSHDUT
38372,C0035229,Respiratoire insufficiëntie,SY,MSHDUT
38373,C0035229,insufficiëntie; ademhaling,PT,ICPC2ICD10DUT
38374,C0035229,insufficiëntie; respiratoir,PT,ICPC2ICD10DUT
38375,C0035229,respiratoir; insufficiëntie,PT,ICPC2ICD10DUT
38376,C0035229,longinsufficiëntie,LLT,MDRDUT
175955,C0035229,Ademhalingsinsufficiëntie,MH,MSHDUT
175956,C0035229,"Respiratoire insufficiëntie, niet gespecificeerd",PT,ICD10DUT


,cui,str,tty,sab
175957,C1145670,respiratoir falen,LLT,MDRDUT
175958,C1145670,respiratoir falen,LLT,MDRDUT
175959,C1145670,respiratoir falen,LLT,MDRDUT
175960,C1145670,respiratoir falen,PT,MDRDUT


## Filter on terminology and type

In [12]:
dutch_umls_original.sab.unique()

array(['MSHDUT', 'ICD10DUT', 'ICPC2ICD10DUT', 'MDRDUT', 'ICPCDUT',
       'ICPC2EDUT', 'LNC-NL-NL'], dtype=object)

In [13]:
# Assess terms per terminology
dutch_umls_original.loc[dutch_umls_original['sab'].isin(['LNC-NL-NL', 'ICPC2ICD10DUT'])].sample(5)

,cui,str,tty,sab
247926,C2350621,mycotisch; mycetoom,PT,ICPC2ICD10DUT
124896,C0542334,neurose; compulsie,PT,ICPC2ICD10DUT
133062,C0745103,Fredrickson; type IIa hyperlipoproteïnemie,PT,ICPC2ICD10DUT
24125,C0022411,gewricht; corpus liberum,PT,ICPC2ICD10DUT
19777,C0018924,gewricht; bloeding,PT,ICPC2ICD10DUT


In [14]:
# 'LNC-NL-NL' and 'ICPC2ICD10DUT' names are not usefull for named entity linking, so we exclude these
dutch_umls_sab_filtered = dutch_umls_original.loc[~dutch_umls_original['sab'].isin(['LNC-NL-NL', 'ICPC2ICD10DUT'])].copy()
dutch_umls_sab_filtered.sample(5)

,cui,str,tty,sab
5505,C0006114,hersenoedeem,LLT,MDRDUT
134923,C0752303,"Manifestatie, urologische",MH,MSHDUT
247338,C2242522,symmetrische T-golf-inversie,LLT,MDRDUT
157815,C0863001,niet-gesplitst folliculair grootcellig kerncellymfoom (Lukes-Collins-classificatie) refractair,LLT,MDRDUT
180917,C1293146,botcyste-excisie,PT,MDRDUT


In [15]:
dutch_umls_sab_filtered

,cui,str,tty,sab
0,C0000696,A-zenuwvezels,MH,MSHDUT
1,C0000715,Abattoir,MH,MSHDUT
2,C0000715,Abattoirs,SY,MSHDUT
3,C0000722,Abbreviated Injury Scale,MH,MSHDUT
4,C0000726,Buik,MH,MSHDUT
...,...,...,...,...
289788,C5578118,ureterale lavage,LLT,MDRDUT
289789,C5578119,vaccintoediening aan huis,LLT,MDRDUT
289790,C5578120,onjuiste vaccintoedieningstechniek,LLT,MDRDUT
289791,C5578121,ontwenningssyndroom na gebruik van lokale steroïden,LLT,MDRDUT


In [16]:
# Assess terms per type
dutch_umls_sab_filtered.loc[dutch_umls_sab_filtered['tty']=='HT'].sample(5)

,cui,str,tty,sab
24341,C0022610,Kernicterus,HT,ICD10DUT
123621,C0524368,beroepsmatige milieuproblemen,HT,MDRDUT
64384,C0162835,hypopigmentatieaandoeningen,HT,MDRDUT
141735,C0836924,trombocytosen,HT,MDRDUT
166788,C0947903,binnenoor therapeutische verrichtingen,HT,MDRDUT


## Term type in source
Some source-defined term types are not relevant for our use case. In the next part we will drop those. See https://www.nlm.nih.gov/research/umls/knowledge_sources/metathesaurus/release/abbreviations.html 

In [17]:
dutch_umls_sab_filtered.tty.value_counts()

tty
LLT    75787
PT     36587
MH     28600
SY     11857
HT      3295
HG       337
SMQ      230
CP        38
OS        27
AB        27
Name: count, dtype: int64

| TTY  | Description | Example |
| - | - | - |
| LLT | Lower Level Term | heupkombreuk, buikkramp|
| PT | Designated preferred name | harthypertrofie, Pancoast-syndroom |
| MH | Main heading | Dehydratie, Astma |
| SY | Designated synonym | Spanningshoofdpijn, Ziekte van Hodgkin |
| HT | Hierarchical term | calciummetabolismestoornissen, oculaire hemorragische aandoeningen |
| HG | High Level Group Term  | complicaties geassocieerd met medisch hulpmiddel, zuur-basestoornissen |
| SMQ| Standardised MedDRA Query | Leveraandoeningen (SMQ) , Tumormarkers (SMQ) |
| CP | ICPC component process (in original form) | Ander bloedonderzoek, Medicatie/recept/injectie |
| AB | Abbreviation in any source vocabulary | Infec, Neopl, Ear, Endo |
| OS | System-organ class | Bloed- en lymfestelselaandoeningen, Infecties en parasitaire aandoeningen |

In [18]:
# Select a set of TTYs that seem most relevant for named entity recognition
tty_selection = ['PT', 'LLT', 'MH', 'SY']
dutch_umls_tty_filtered = dutch_umls_sab_filtered.loc[dutch_umls_sab_filtered.tty.isin(tty_selection)].copy()

# Keep only relevant columns
dutch_umls_tty_filtered = dutch_umls_tty_filtered[['cui', 'str', 'tty', 'sab']]
dutch_umls_tty_filtered.sample(5)

,cui,str,tty,sab
43719,C0039990,Thoracostomie,MH,MSHDUT
76755,C0243001,Abdominaal abces,SY,MSHDUT
244499,C1963921,chemische flebitis,PT,MDRDUT
75887,C0242379,longneoplasma maligne,LLT,MDRDUT
15820,C0015726,vrees,LLT,MDRDUT


## Preferred/pretty/primary names
For MedCAT, and other named entity linking methods, it's useful to designate a single name as the preferred name, also sometimes called primary name or pretty name. This name can be presented to the end-user in webapplications, and should therefore be the most descriptive and commonly used name. All other names (synonyms, abreviations and common mispellings) will then be considered synonyms. MedCAT used 'P' as the value for preferred name, and all other names should have the value 'A' (see https://github.com/CogStack/MedCAT/blob/master/examples/README.md).

Most, if not all, of UMLS concepts have a preferred in English. For other languages,
it can be difficult to select a preferred name, because each source vocabulary has one or
multiple preferred names for a concepts.

It's not possible to keep the English UMLS preferred names, because MedCAT would add those names to the concept table for entity linking. Perhaps future functionality can be added for MedCAT to prevent taking these preferred names into account during entity linking.

### Solution 1: Use UMLS source vocabularies preferred names
For a rough but effective solution to get decent preferred names for the Dutch terms, change the terms that have the value "Designated preferred name" (PT) for the "Term Type in Source" (TTY) to MedCAT's preferred name value (P), and all others can be saved as (A). This leads to many concepts having multiple preferred names.

In [19]:
# dutch_umls_tty_filtered.tty.replace({'PT': 'P',
#                                      'LLT': 'A',
#                                      'MH': 'A',
#                                      'SY': 'A'}, inplace=True)

### Solution 2: Use preferred names from Dutch SNOMED
In previous experiments we have shown that the Dutch vocabularies from UMLS and Dutch SNOMED complement each other. SNOMED provides most of the names, and contains excellent primary names. So we could use the preferred names from Dutch SNOMED, and for the terms not in that vocabulary, let MedCAT pick a random one.

In [20]:
# Drop tty column, put it back in just before merging with SNOMED
dutch_umls_tty_filtered.drop(['tty'], axis=1, inplace=True)

## Clean values
ICD10DUT and MDRDUT contain names that are more definitions than then how they would be found in text. For example, "colonkanker NAO" (see https://alt.meddra.org/files_acrobat/intguide_25_0_Dutch.pdf) and "Aandoening van ooglid, niet gespecificeerd". Names are more useful for named entity recongition when the descriptive part is removed.

In [21]:
dutch_umls_clean = dutch_umls_tty_filtered.copy()

# Remove ' nao' and ' NAO'
print(f"Number of terms containing ' NAO': {len(dutch_umls_clean.loc[dutch_umls_clean['str'].str.contains(' NAO')])}")
print(f"Number of terms containing ' nao': {len(dutch_umls_clean.loc[dutch_umls_clean['str'].str.contains(' nao')])}")
dutch_umls_clean.str = dutch_umls_clean.str.replace({' NAO': '', ' \(NAO\)': '', ' nao': ''}, regex=True)

Number of terms containing ' NAO': 3244
Number of terms containing ' nao': 32


In [22]:
print(f"Number of terms containing 'gespecificeerd': {len(dutch_umls_clean.loc[dutch_umls_clean['str'].str.contains('gespecificeerd')])}")

# Remove suffix
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerd')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerd onderzoek')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerde graad')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerde oorzaak')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerde plaats')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerd type')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerd deel')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerd gebruik')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerd naar behandelperiode')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerd naar betrokkenheid')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerde toestand')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet-gespecificeerd naar oorzaak')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix(', niet gespecificeerd')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removesuffix('; Niet gespecificeerd')

# Remove prefix
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removeprefix('niet-gespecificeerde ')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removeprefix('niet-gespecificeerd ')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removeprefix('Niet gespecificeerd ')
dutch_umls_clean['str'] = dutch_umls_clean['str'].str.removeprefix('Niet gespecificeerde ')

print(f"Number of terms containing 'gespecificeerd': {len(dutch_umls_clean.loc[dutch_umls_clean['str'].str.contains('gespecificeerd')])}")

Number of terms containing 'gespecificeerd': 4609
Number of terms containing 'gespecificeerd': 2503


There are many occurences of 'gespecificeerd' in names left, but these are more difficult to clean. For now, these are likely not causing any issues (but probably also not adding much) so we'll keep them in our table.

In [23]:
# Convert title-formatted names to lowercase
dutch_umls_clean['str'] = dutch_umls_clean['str'].apply(convert_title_to_lowercase, split_char=' ')
dutch_umls_clean['str'] = dutch_umls_clean['str'].apply(convert_title_to_lowercase, split_char='-')

In [24]:
# Drop duplicates
print(f'Records before dropping duplicates: {dutch_umls_clean.shape[0]}')
dutch_umls_clean = dutch_umls_clean.drop_duplicates(subset=['cui', 'str', 'sab'], keep='first').reset_index(drop=True)
print(f'Records after dropping duplicates: {dutch_umls_clean.shape[0]}')

Records before dropping duplicates: 152831
Records after dropping duplicates: 118269


In [25]:
dutch_umls_clean[dutch_umls_clean.cui == 'C0002736']

,cui,str,sab
1278,C0002736,"sclerose, laterale, amyotrofische (ALS)",MSHDUT
1279,C0002736,ALS,MDRDUT
1280,C0002736,ALS,MSHDUT
1281,C0002736,amyotrofische laterale sclerose (ALS),MSHDUT
1282,C0002736,lou-gehrig-ziekte,MSHDUT
1283,C0002736,motoneuronziekte,MSHDUT
1284,C0002736,"sclerose, amyotrofische laterale",MSHDUT
1285,C0002736,amyotrofe laterale sclerose,MDRDUT


## Merge rows from different vocabularies

In [27]:
dutch_umls = dutch_umls_clean.copy()

# Merge SAB into single row
print(f'Records before merging rows: {dutch_umls.shape[0]}')
dutch_umls = dutch_umls.groupby(['cui','str'])['sab'].apply('|'.join).reset_index()
print(f'Records after merging rows: {dutch_umls.shape[0]}')
dutch_umls[dutch_umls.cui == 'C0002736']

Records before merging rows: 118269
Records after merging rows: 113526


,cui,str,sab
1137,C0002736,ALS,MDRDUT|MSHDUT
1138,C0002736,amyotrofe laterale sclerose,MDRDUT
1139,C0002736,amyotrofische laterale sclerose (ALS),MSHDUT
1140,C0002736,lou-gehrig-ziekte,MSHDUT
1141,C0002736,motoneuronziekte,MSHDUT
1142,C0002736,"sclerose, amyotrofische laterale",MSHDUT
1143,C0002736,"sclerose, laterale, amyotrofische (ALS)",MSHDUT


In [28]:
# Add tty column with value 'A' to set these names as synonyms 
dutch_umls['tty'] = 'A'
dutch_umls.head()

,cui,str,sab,tty
0,C0000696,a-zenuwvezels,MSHDUT,A
1,C0000715,abattoir,MSHDUT,A
2,C0000715,abattoirs,MSHDUT,A
3,C0000722,abbreviated injury scale,MSHDUT,A
4,C0000726,abdomen,MSHDUT,A


# Add Dutch names from SNOMED
UMLS does not contain the Dutch SNOMEDCT, but it does contain the English SNOMEDCT. So through the English SNOMED concepts, we can map the Dutch SNOMED names to UMLS.

Dutch SNOMED names with SNOMED ID **->** Get English SNOMED ID to UMLS ID mapping **->** Map Dutch SNOMED names with SNOMED ID to UMLS ID

### Load SNOMED US

In [29]:
query = "SELECT distinct cui, scui FROM MRCONSO where sab = 'SNOMEDCT_US'"
snomed_us = pd.read_sql_query(query, con=connection)
snomed_us.scui = snomed_us.scui.astype(str)
print(f'SNOMED US terms with UMLS CUI: {snomed_us.shape[0]}')
snomed_us.head()

SNOMED US terms with UMLS CUI: 367227


,cui,scui
0,C0000052,58488005
1,C0000097,285407008
2,C0000102,13579002
3,C0000163,112116001
4,C0000167,46120009


### Load SNOMED NL
We're using a cleaned and filtered list of Dutch SNOMED names, see other notebook in this repository how this is created.

In [30]:
snomed_dutch = pd.read_csv(snomed_dutch_file, dtype='str')
snomed_dutch.head()

,cui,str,tty,tui,sab
0,104001,excisie van afwijkend weefsel van patella,P,verrichting,SNOMEDCT_NL
1,104001,excisie van laesie van knieschijf,A,verrichting,SNOMEDCT_NL
2,106004,structuur van posterieure carpale regio,P,lichaamsstructuur,SNOMEDCT_NL
3,106004,posterieur gebied van handwortel,A,lichaamsstructuur,SNOMEDCT_NL
4,106004,posterieur carpaal gebied,A,lichaamsstructuur,SNOMEDCT_NL


In [31]:
snomed_dutch.shape

(533017, 5)

## Find ambiguous mapping

First find which SNOMED concepts can map to UMLS concepts. SNOMED concepts can map to multiple UMLS concepts.

In [32]:
# Create SNOMED - UMLS mapping
snomed_to_umls_mapping = snomed_us.groupby('scui')['cui'].apply(list).to_dict()
print(f'Number of SNOMED concepts that map to at least 1 UMLS concept: {len(snomed_to_umls_mapping)}')

Number of SNOMED concepts that map to at least 1 UMLS concept: 365049


In [33]:
# Check ambiguity of UMLS-SNOMED mapping
unambiguous_mapping_ids = set()
ambiguous_mapping_ids = set()
for snomed_id in snomed_to_umls_mapping:
    if len(snomed_to_umls_mapping[snomed_id]) == 1:
        unambiguous_mapping_ids.add(snomed_id)
    else:
        ambiguous_mapping_ids.add(snomed_id)
print(f'Number of SNOMED concepts that map to only 1 concept: {len(unambiguous_mapping_ids)}')
print(f'Number of SNOMED concepts that map to multiple concepts: {len(ambiguous_mapping_ids)}')

Number of SNOMED concepts that map to only 1 concept: 363089
Number of SNOMED concepts that map to multiple concepts: 1960


So 2073 SNOMED concepts map to multiple UMLS concepts. If the Dutch names from these concepts are added to the UMLS concept table, it will introduce ambiguity, which could lead to problems in our downstream named entity linking methods. Therefore, for now, this method does not add names for these ambiguously mapping SNOMED concepts.

## Example of ambiguous mapping
This section illustrates the ambiguous mapping problem

In [34]:
# Find example
ambiguous_mapping_ids = [int(code) for code in ambiguous_mapping_ids]
ambiguous_mapping_ids.sort()
ambiguous_mapping_ids = [str(code) for code in ambiguous_mapping_ids]
ambiguous_mapping_ids[0:5]

['115006', '216004', '289002', '344001', '489004']

In [35]:
query = "SELECT distinct cui, code, str FROM MRCONSO where sab = 'SNOMEDCT_US' and CODE = '216004'"
snomed_us_example = pd.read_sql_query(query, con=connection)
snomed_us_example.head()

,cui,code,str
0,C0151836,216004,Paranoid reaction
1,C1704268,216004,Delusion of persecution
2,C1704268,216004,Persecutory delusion
3,C1704268,216004,Delusion of persecution (finding)


In [36]:
snomed_dutch[snomed_dutch.cui == '216004']

,cui,str,tty,tui,sab
124,216004,achtervolgingswaan,P,bevinding,SNOMEDCT_NL


In [37]:
dutch_umls[dutch_umls.cui.isin(['C0151836', 'C1704268'])]

,cui,str,sab,tty
34557,C0151836,paranoïde reactie,MDRDUT,A
34558,C0151836,reactie paranoïde,MDRDUT,A
97890,C1704268,vervolgingswaan,MDRDUT,A
97891,C1704268,waan van achtervolging,MDRDUT,A


So SNOMED US has four names for 216004. Three of these names map to C1704268 and one maps to C0151836. In SNOMED NL, there is only one name for this concept. In our current UMLS table, we have 2 names for each UMLS concept. We could map the SNOMED name to both concepts (1), to a specific one (2), or skip it (3):

1. Mapping to both will cause ambiguity. It could have an effect on entity linking, or it could be solved during MedCAT's disambiguation functionality based on unsupervised training, but that depends on the synonyms and their presence in the training corpus. In this example there is only 1 Dutch SNOMED term, but when there are multiple Dutch SNOMED terms, adding all to both terms will lead to many duplicates.
2. Mapping to a single one is the best option for a single example, but this requires manual curreation, is time consuming, not within the scope of this project and can be quite difficult. There are about 2000 of these terms. SNOMED US names are in UMLS, so those ambiguously mapping names are added by either the UMLS or SNOMED team. Perhaps in future versions, this is corrected at either UMLS or SNOMED level.
3. Skipping the name is the easiest option and will not lead to potential difficult downstream interpretation. The drawback is that the name, which in this example is not in any other Dutch ontology, will not be in the Dutch UMLS concept table.

Currently, approach #3 is used.

In [38]:
# Another example of ambiguous SNOMED-CT -> UMLS mapping:
snomed_dutch[snomed_dutch.cui == '2776000']

,cui,str,tty,tui,sab
2823,2776000,delier,P,aandoening,SNOMEDCT_NL
2824,2776000,delirium,A,aandoening,SNOMEDCT_NL


In [39]:
# Show that a single SNOMED ID maps to multiple UMLS concepts
query = "SELECT distinct cui, code, str FROM MRCONSO where sab = 'SNOMEDCT_US' and CODE = '2776000'"
snomed_us_example = pd.read_sql_query(query, con=connection)
snomed_us_example.head()

,cui,code,str
0,C0011206,2776000,Delirium
1,C0011206,2776000,Acute brain syndrome
2,C0011206,2776000,ABS - Acute brain syndrome
3,C0011206,2776000,Delirium (disorder)
4,C0029221,2776000,Organic brain syndrome


## Merge SNOMED Dutch with UMLS Dutch

In [40]:
def map_dutch_snomed_to_umls(row):
    snomed_id = row['cui']
    if snomed_id in unambiguous_mapping_ids:
        cui = snomed_to_umls_mapping[snomed_id][0]
        snomed_names_to_add.append([cui, row['str'], row['tty']])
    else:
        snomed_names_to_skip.append([snomed_id, row['str'], row['tty']])
        
# Create lists to fill with SNOMED names and their UMLS CUIs
snomed_names_to_add = list()
snomed_names_to_skip = list()

# Map Dutch SNOMED to UMLS
snomed_dutch.apply(map_dutch_snomed_to_umls, axis = 1)

print(f'Number of Dutch names in existing UMLS table: {dutch_umls.shape[0]}')
print(f'Number of Dutch SNOMED names to add: {len(snomed_names_to_add)}')
print(f'Number of Dutch SNOMED names to skip: {len(snomed_names_to_skip)}')

Number of Dutch names in existing UMLS table: 113526
Number of Dutch SNOMED names to add: 480552
Number of Dutch SNOMED names to skip: 52465


In [41]:
# Format SNOMED names in pandas dataframe
snomed_names_with_cui = pd.DataFrame(snomed_names_to_add, columns = ['cui', 'str', 'tty'])
snomed_names_with_cui['sab'] = 'SNOMEDCT_NL'
snomed_names_with_cui.head()

,cui,str,tty,sab
0,C0187893,excisie van afwijkend weefsel van patella,P,SNOMEDCT_NL
1,C0187893,excisie van laesie van knieschijf,A,SNOMEDCT_NL
2,C0230364,structuur van posterieure carpale regio,P,SNOMEDCT_NL
3,C0230364,posterieur gebied van handwortel,A,SNOMEDCT_NL
4,C0230364,posterieur carpaal gebied,A,SNOMEDCT_NL


### Remove duplicate SNOMED concepts
Earlier the problem was discussed of a SNOMED term that maps to multiple UMLS concepts. There's also the problem of snomed names that are ambiguous in SNOMED itself. Some of these, like "abces" are not ambiguous in UMLS. So when mapping these concepts to UMLS, the ambiguity is solved.

In [42]:
snomed_dutch[snomed_dutch.str == 'abces']

,cui,str,tty,tui,sab
48318,44132006,abces,P,afwijkende morfologie,SNOMEDCT_NL
134172,128477000,abces,P,aandoening,SNOMEDCT_NL


In [43]:
snomed_to_umls_mapping['44132006']

['C0000833']

In [44]:
snomed_to_umls_mapping['128477000']

['C0000833']

In [45]:
snomed_names_with_cui[snomed_names_with_cui.cui == 'C0000833']

,cui,str,tty,sab
45577,C0000833,abces,P,SNOMEDCT_NL
127674,C0000833,abces,P,SNOMEDCT_NL


In [46]:
print(f'Number of SNOMED concepts that include names that are ambiguous in SNOMED: {snomed_names_with_cui.shape[0]}')
snomed_names_with_cui = snomed_names_with_cui.drop_duplicates(subset=['cui', 'str', 'sab', 'tty'], keep='first').reset_index(drop=True)
print(f'Number of SNOMED names: {snomed_names_with_cui.shape[0]}')
snomed_names_with_cui[snomed_names_with_cui.cui == 'C0000833']

Number of SNOMED concepts that include names that are ambiguous in SNOMED: 480552
Number of SNOMED names: 479703


,cui,str,tty,sab
45574,C0000833,abces,P,SNOMEDCT_NL


### Clean SNOMED names
Some names from SNOMED are also in Title-format, such as ziekte van Parkinson. To prevent duplication, lowercase these terms.

In [47]:
display(snomed_names_with_cui[snomed_names_with_cui.cui.isin(['C0030567', 'C0002736'])])

,cui,str,tty,sab
50833,C0030567,ziekte van Parkinson,P,SNOMEDCT_NL
50834,C0030567,paralysis agitans,A,SNOMEDCT_NL
89569,C0002736,amyotrofische laterale sclerose,P,SNOMEDCT_NL
89570,C0002736,ALS,A,SNOMEDCT_NL


In [48]:
snomed_names_with_cui['str'] = snomed_names_with_cui['str'].apply(convert_title_to_lowercase, split_char=' ')
snomed_names_with_cui['str'] = snomed_names_with_cui['str'].apply(convert_title_to_lowercase, split_char='-')

In [49]:
# Examples
display(snomed_names_with_cui[snomed_names_with_cui.cui.isin(['C0030567', 'C0002736'])])

,cui,str,tty,sab
50833,C0030567,ziekte van parkinson,P,SNOMEDCT_NL
50834,C0030567,paralysis agitans,A,SNOMEDCT_NL
89569,C0002736,amyotrofische laterale sclerose,P,SNOMEDCT_NL
89570,C0002736,ALS,A,SNOMEDCT_NL


### Concatenate SNOMED names to UMLS table

In [50]:
# Add SNOMED names to UMLS
dutch_umls_snomed = pd.concat([dutch_umls, snomed_names_with_cui])
print(f'Number of Dutch names in UMLS + SNOMED table: {dutch_umls_snomed.shape[0]}')

dutch_umls_snomed.sort_values(by=['cui', 'tty', 'sab', 'str'], ascending=[True, False, True, True], inplace=True)
dutch_umls_snomed.loc[dutch_umls_snomed.cui == 'C0030567']

Number of Dutch names in UMLS + SNOMED table: 593229


,cui,str,sab,tty
50833,C0030567,ziekte van parkinson,SNOMEDCT_NL,P
21717,C0030567,ziekte van parkinson,ICD10DUT|MSHDUT|MDRDUT,A
21715,C0030567,parkinson-syndroom,MDRDUT,A
21712,C0030567,idiopathische ziekte van parkinson,MSHDUT,A
21713,C0030567,lewy-body-ziekte,MSHDUT,A
21716,C0030567,"parkinsonisme, primair",MSHDUT,A
21714,C0030567,paralysis agitans,MSHDUT|MDRDUT,A
50834,C0030567,paralysis agitans,SNOMEDCT_NL,A


In [51]:
# Grouping rows on SAB
dutch_umls_snomed = dutch_umls_snomed.groupby(['cui', 'str'], as_index=False).agg({'sab' : '|'.join, 'tty' : '|'.join}).copy()

# Clean tty column
dutch_umls_snomed.tty = dutch_umls_snomed.tty.apply(clean_name_status_column)
dutch_umls_snomed.sort_values(by=['cui', 'tty'], ascending=[True, False], inplace=True)
dutch_umls_snomed.reset_index(drop=True,inplace=True)
print(f'Number of names after merging rows on SAB: {dutch_umls_snomed.shape[0]}')

# Check example
dutch_umls_snomed.loc[dutch_umls_snomed.cui == 'C0030567']

Number of names after merging rows on SAB: 582007


,cui,str,sab,tty
29224,C0030567,ziekte van parkinson,SNOMEDCT_NL|ICD10DUT|MSHDUT|MDRDUT,P
29225,C0030567,idiopathische ziekte van parkinson,MSHDUT,A
29226,C0030567,lewy-body-ziekte,MSHDUT,A
29227,C0030567,paralysis agitans,MSHDUT|MDRDUT|SNOMEDCT_NL,A
29228,C0030567,parkinson-syndroom,MDRDUT,A
29229,C0030567,"parkinsonisme, primair",MSHDUT,A


## Remove problematic names


In [52]:
names_to_remove = ['bij', # C0004923
                   'bijen', # C0004923
                   'haar', # C0018494
                   'bleek', # C0678215
                   'weer', # C0043085
                   'na+'] # C0337443
dutch_umls_snomed[dutch_umls_snomed.str.isin(names_to_remove)]

,cui,str,sab,tty
3692,C0004923,bij,MSHDUT,A
3693,C0004923,bijen,MSHDUT,A
16652,C0018494,haar,MSHDUT|SNOMEDCT_NL,A
41242,C0043085,weer,MSHDUT,A
157814,C0337443,na+,MDRDUT,A
229540,C0444095,haar,SNOMEDCT_NL,P
311398,C0678215,bleek,MDRDUT,A
462841,C3245527,haar,SNOMEDCT_NL,P


In [53]:
# Remove rows
rows_to_remove = dutch_umls_snomed[dutch_umls_snomed.str.isin(names_to_remove)].index
print(f'Number of rows before removing rows: {dutch_umls_snomed.shape[0]}')
dutch_umls_snomed = dutch_umls_snomed.drop(dutch_umls_snomed.index[rows_to_remove])
print(f'Number of rows after removing rows: {dutch_umls_snomed.shape[0]}')

Number of rows before removing rows: 582007
Number of rows after removing rows: 581999


## Add custom CUIs
Sometimes names or concept are not captured in any of the Dutch terminologies. By looking up the English name for these concepts, we can add custom Dutch names using the real UMLS identifier.

In [54]:
dutch_umls_snomed.head()

,cui,str,sab,tty
0,C0000052,"1,4-alfa-glucaan vertakkend enzym",SNOMEDCT_NL,P
1,C0000097,methyl-fenyltetrahydropyridine,SNOMEDCT_NL,P
2,C0000097,"1-methyl-4-fenyl-1,2,3,6-tetrahydropyridine",SNOMEDCT_NL,A
3,C0000097,MPTP,SNOMEDCT_NL,A
4,C0000172,18-hydroxycorticosteron,SNOMEDCT_NL,P


In [55]:
custom_concepts = pd.read_csv(custom_concepts_file)
custom_concepts

,cui,str,tty,sab
0,C0456984,uitslag,A,UMCU
1,C0019080,bloedt,A,UMCU
2,C0019080,bloeden,A,UMCU
3,C0225844,RA,A,UMCU
4,C0225883,RV,A,UMCU
5,C0225897,LV,A,UMCU
6,C0011206,delier,P,UMCU
7,C0003232,antibiotica,P,UMCU


In [56]:
print(f'Number of rows before adding rows: {dutch_umls_snomed.shape[0]}')
dutch_umls_snomed = pd.concat([dutch_umls_snomed, custom_concepts])
print(f'Number of rows after adding rows: {dutch_umls_snomed.shape[0]}')

Number of rows before adding rows: 581999
Number of rows after adding rows: 582007


In [57]:
dutch_umls_snomed.head()

,cui,str,sab,tty
0,C0000052,"1,4-alfa-glucaan vertakkend enzym",SNOMEDCT_NL,P
1,C0000097,methyl-fenyltetrahydropyridine,SNOMEDCT_NL,P
2,C0000097,"1-methyl-4-fenyl-1,2,3,6-tetrahydropyridine",SNOMEDCT_NL,A
3,C0000097,MPTP,SNOMEDCT_NL,A
4,C0000172,18-hydroxycorticosteron,SNOMEDCT_NL,P


## Add TUI (types)
UMLS concepts have one or multiple types. These types are kept in a separate table, `MRSTY`. See https://semanticnetwork.nlm.nih.gov/download/SemGroups.txt for all types.

In [58]:
# Load TUI table from MySQL
query = """
SELECT cui, tui FROM MRSTY
"""
tui_original = pd.read_sql_query(query, con=connection)
tui_original.head()

,cui,tui
0,C0000039,T109
1,C0000039,T121
2,C0000052,T116
3,C0000052,T126
4,C0000097,T109


In [59]:
# Add TUI column to UMLS + SNOMED CUI table
dutch_umls_snomed = dutch_umls_snomed.merge(tui_original, how='left', on='cui')
dutch_umls_snomed.head(20)

,cui,str,sab,tty,tui
0,C0000052,"1,4-alfa-glucaan vertakkend enzym",SNOMEDCT_NL,P,T116
1,C0000052,"1,4-alfa-glucaan vertakkend enzym",SNOMEDCT_NL,P,T126
2,C0000097,methyl-fenyltetrahydropyridine,SNOMEDCT_NL,P,T109
3,C0000097,methyl-fenyltetrahydropyridine,SNOMEDCT_NL,P,T131
4,C0000097,"1-methyl-4-fenyl-1,2,3,6-tetrahydropyridine",SNOMEDCT_NL,A,T109
5,C0000097,"1-methyl-4-fenyl-1,2,3,6-tetrahydropyridine",SNOMEDCT_NL,A,T131
6,C0000097,MPTP,SNOMEDCT_NL,A,T109
7,C0000097,MPTP,SNOMEDCT_NL,A,T131
8,C0000172,18-hydroxycorticosteron,SNOMEDCT_NL,P,T109
9,C0000172,18-hydroxycorticosteron,SNOMEDCT_NL,P,T121


In [61]:
dutch_umls_snomed

,cui,str,sab,tty,tui
0,C0000052,"1,4-alfa-glucaan vertakkend enzym",SNOMEDCT_NL,P,T116
1,C0000052,"1,4-alfa-glucaan vertakkend enzym",SNOMEDCT_NL,P,T126
2,C0000097,methyl-fenyltetrahydropyridine,SNOMEDCT_NL,P,T109
3,C0000097,methyl-fenyltetrahydropyridine,SNOMEDCT_NL,P,T131
4,C0000097,"1-methyl-4-fenyl-1,2,3,6-tetrahydropyridine",SNOMEDCT_NL,A,T109
...,...,...,...,...,...
591361,C0225844,RA,UMCU,A,T023
591362,C0225883,RV,UMCU,A,T023
591363,C0225897,LV,UMCU,A,T023
591364,C0011206,delier,UMCU,P,T048


## TUI Filtering
What types of concepts (TUIs) should be removed depends on the domain and question of subsequent analysis. After assessing performance of named entity linking of these names on a few clinical documents, our team dediced to remove the following TUIs.

SNOMED names are already filtered in a seperate filtering step based on type, which is done in the notebook that creates the SNOMED concept table.

In [62]:
tuis_to_remove = [
    
    # Concepts & Ideas
    'T078', # Idea or Concept
    'T089', # Regulation or Law

    # Living beings
    'T011', # Amphibian
    'T008', # Animal
    'T012', # Bird
    'T013', # Fish
    'T015', # Mammal
    'T001', # Organism
    'T001', # Plant
    'T014', # Reptile
    'T010', # Vertebrate
    
    # Objects
    'T168', # Food
    
    # Organizations
    'T093', # Healthcare Related Organization
    
    # Geographic areas
    'T083', # Geographic Area
    
    # Added
    'T056', # Daily or Recreational Activity
    'T065', # Educational Activity
    'T064', # Governmental or Regulatory Activity
    'T170', # Intellectual Product
    'T171', # Language
    'T073', # Manufactured Object
    'T057', # Occupational Activity
    'T090', # Occupation or Discipline
    'T066', # Machine Activity
    'T092', # Organization
    'T072', # Physical Object
    'T067', # Phenomenon or Process
    'T097', # Professional or Occupational Group
    'T094', # Professional Society
    'T080', # Qualitative Concept
    'T081', # Quantitative Concept
    'T095', # Self-help or Relief Organization
    'T082', # Spatial Concept
    'T079', # Temporal Concept
]
                  
dutch_umls_snomed[dutch_umls_snomed.tui.isin(tuis_to_remove)].head()

,cui,str,sab,tty,tui
43,C0000715,abattoir,MSHDUT,A,T073
44,C0000715,abattoirs,MSHDUT,A,T073
45,C0000722,abbreviated injury scale,MSHDUT,A,T170
232,C0000858,samenvatting en indexering,MSHDUT,A,T057
233,C0000863,abu dhabi,MSHDUT,A,T083


In [64]:
# Remove rows based on TUI
rows_to_remove = dutch_umls_snomed[dutch_umls_snomed.tui.isin(tuis_to_remove)].index
print(f'Number of rows before removing rows: {dutch_umls_snomed.shape[0]}')
dutch_umls_snomed = dutch_umls_snomed.drop(dutch_umls_snomed.index[rows_to_remove])
print(f'Number of rows after removing rows: {dutch_umls_snomed.shape[0]}')

Number of rows before removing rows: 581920
Number of rows after removing rows: 581920


In [67]:
dutch_umls_snomed = dutch_umls_snomed.groupby(['cui', 'str', 'tty', 'sab'])['tui'].apply('|'.join).reset_index()
print(f'Number of rows after merging TUIs in single value: {dutch_umls_snomed.shape[0]}')

Number of rows after merging TUIs in single value: 572898


In [68]:
dutch_umls_snomed

,cui,str,tty,sab,tui
0,C0000052,"1,4-alfa-glucaan vertakkend enzym",P,SNOMEDCT_NL,T116|T126
1,C0000097,"1-methyl-4-fenyl-1,2,3,6-tetrahydropyridine",A,SNOMEDCT_NL,T109|T131
2,C0000097,MPTP,A,SNOMEDCT_NL,T109|T131
3,C0000097,methyl-fenyltetrahydropyridine,P,SNOMEDCT_NL,T109|T131
4,C0000172,18-hydroxycorticosteron,P,SNOMEDCT_NL,T109|T121
...,...,...,...,...,...
572893,C5689081,gemetastaseerd plaveiselcelcarcinoom in lymfeklier van bovenste extremiteit,P,SNOMEDCT_NL,T191
572894,C5689081,gemetastaseerd plaveiselcelcarcinoom in lymfeklier van bovenste ledemaat,A,SNOMEDCT_NL,T191
572895,C5689082,gemetastaseerd plaveiselcelcarcinoom in lymfeklier van borstkas,A,SNOMEDCT_NL,T191
572896,C5689082,gemetastaseerd plaveiselcelcarcinoom in lymfeklier van thorax,P,SNOMEDCT_NL,T191


### Update column names
In MedCAT v1.0 the column name specification has changed and is defined as in the [README.md in examples](https://github.com/CogStack/MedCAT/tree/master/examples).

In [69]:
dutch_umls_snomed.rename(columns={'str': 'name', 'tty': 'name_status', 'sab': 'ontologies', 'tui': 'type_ids'}, inplace=True)
dutch_umls_snomed.sort_values(by=['cui', 'name_status'], ascending=[True, False], inplace=True)
dutch_umls_snomed.head()

,cui,name,name_status,ontologies,type_ids
0,C0000052,"1,4-alfa-glucaan vertakkend enzym",P,SNOMEDCT_NL,T116|T126
3,C0000097,methyl-fenyltetrahydropyridine,P,SNOMEDCT_NL,T109|T131
1,C0000097,"1-methyl-4-fenyl-1,2,3,6-tetrahydropyridine",A,SNOMEDCT_NL,T109|T131
2,C0000097,MPTP,A,SNOMEDCT_NL,T109|T131
4,C0000172,18-hydroxycorticosteron,P,SNOMEDCT_NL,T109|T121


### Save output

In [60]:
# Print statistics
print(f'Unique concepts: {len(dutch_umls_snomed.cui.unique())}')
print(f'Unique names: {len(dutch_umls_snomed.name.unique())}')
print(f'Ambiguous names: {len(dutch_umls_snomed) - len(dutch_umls_snomed.name.unique())}')
print(f'Total names in concepts: {len(dutch_umls_snomed)}')

# Save final concept table
dutch_umls_snomed.to_csv(output_file, index=False)

Unique concepts: 253215
Unique names: 568304
Ambiguous names: 4594
Total names in concepts: 572898


## Add drug names
Only run this part below if you want to further expand the concept database with drug names, which adds around 270k lines. Many drugs only have an international name, or use the international name more often than the Dutch name, so adding these from ATC, Drugbank and RXNorm can be a good addition to the concept table. 

After assessing the resulting list it will be clear that many names will not be useful in named entity recognition, because they will probably never be used in natural language.

In [61]:
#In case you want to begin from here, load existing concept table:
#dutch_umls_snomed = pd.read_csv("04_ConceptDB/umls-dutch_{UMLS_DUTCH_VERSION}.csv", dtype=str)

In [62]:
# Retrieve Dutch UMLS concepts
query = """
SELECT distinct MRCONSO.cui, str as name, sab as ontologies 
FROM MRCONSO
WHERE SAB in ('ATC','DRUGBANK','RXNORM')
"""
drugs_original = pd.read_sql_query(query, con=connection)
drugs_original.head()

,cui,name,ontologies
0,C0000039,"1,2-dipalmitoylphosphatidylcholine",RXNORM
1,C0000266,Parlodel,RXNORM
2,C0000294,mesna,ATC
3,C0000294,mesna,RXNORM
4,C0000378,droxidopa,ATC


In [63]:
# Convert title format to lowercase
drugs_original['name'] = drugs_original['name'].apply(convert_title_to_lowercase, split_char=' ')
drugs_original['name'] = drugs_original['name'].apply(convert_title_to_lowercase, split_char='-')

In [64]:
drugs_original[drugs_original.cui =='C0020740']

,cui,name,ontologies
6073,C0020740,ibuprofen,ATC
6074,C0020740,ibuprofen,RXNORM
6075,C0020740,ibuprofen,DRUGBANK
6076,C0020740,(±)-ibuprofen,DRUGBANK
6077,C0020740,(±)-p-isobutylhydratropic acid,DRUGBANK
6078,C0020740,2-(4-isobutylphenyl)propanoic acid,DRUGBANK
6079,C0020740,(RS)-ibuprofen,DRUGBANK
6080,C0020740,ibuprophen,DRUGBANK
6081,C0020740,(±)-2-(p-isobutylphenyl)propionic acid,DRUGBANK
6082,C0020740,4-isobutylhydratropic acid,DRUGBANK


In [65]:
# Use previously defined preferred names
drugs_original['name_status'] = 'A'

In [66]:
# Merge drugs dataframe with umls_snomed dataframe
dutch_umls_snomed_drugs = pd.concat([dutch_umls_snomed, drugs_original], axis=0)

print("UMLS & SNOMED records: ", len(dutch_umls_snomed))
print("Drug name records: ", len(drugs_original))
print("Combined: ", len(dutch_umls_snomed_drugs))

dutch_umls_snomed_drugs = dutch_umls_snomed_drugs.groupby(['cui', 'name', 'name_status'])['ontologies'].apply('|'.join).reset_index()
print("Records after merging ontologies in single value: ", len(dutch_umls_snomed_drugs))
dutch_umls_snomed_drugs.head()

UMLS & SNOMED records:  572898
Drug name records:  188191
Combined:  761089
Records after merging ontologies in single value:  753536


,cui,name,name_status,ontologies
0,C0000039,"1,2-dipalmitoylphosphatidylcholine",A,RXNORM
1,C0000052,"1,4-alfa-glucaan vertakkend enzym",P,SNOMEDCT_NL
2,C0000097,"1-methyl-4-fenyl-1,2,3,6-tetrahydropyridine",A,SNOMEDCT_NL
3,C0000097,MPTP,A,SNOMEDCT_NL
4,C0000097,methyl-fenyltetrahydropyridine,P,SNOMEDCT_NL


In [67]:
# Add TUI column
dutch_umls_snomed_drugs = dutch_umls_snomed_drugs.merge(tui_original, how='left', on='cui')
print(f'Number of rows containing TUIs: {dutch_umls_snomed_drugs.shape[0]}')

# Remove TUIs that we decided to filter
rows_to_remove = dutch_umls_snomed_drugs[dutch_umls_snomed_drugs.tui.isin(tuis_to_remove)].index
dutch_umls_snomed_drugs = dutch_umls_snomed_drugs.drop(dutch_umls_snomed_drugs.index[rows_to_remove])
print(f'Number of rows filtering TUIs: {dutch_umls_snomed_drugs.shape[0]}')

# Merge TUIs in single value
dutch_umls_snomed_drugs = dutch_umls_snomed_drugs.groupby(['cui', 'name', 'name_status', 'ontologies'])['tui'].apply('|'.join).reset_index()
print(f'Number of rows after merging TUIs in single value: {dutch_umls_snomed_drugs.shape[0]}')

# Rename TUI column to type_ids
dutch_umls_snomed_drugs.rename(columns={'tui': 'type_ids'}, inplace=True)

Number of rows containing TUIs: 812407
Number of rows filtering TUIs: 812146
Number of rows after merging TUIs in single value: 753449


In [68]:
dutch_umls_snomed_drugs.head()

,cui,name,name_status,ontologies,type_ids
0,C0000039,"1,2-dipalmitoylphosphatidylcholine",A,RXNORM,T109|T121
1,C0000052,"1,4-alfa-glucaan vertakkend enzym",P,SNOMEDCT_NL,T116|T126
2,C0000097,"1-methyl-4-fenyl-1,2,3,6-tetrahydropyridine",A,SNOMEDCT_NL,T109|T131
3,C0000097,MPTP,A,SNOMEDCT_NL,T109|T131
4,C0000097,methyl-fenyltetrahydropyridine,P,SNOMEDCT_NL,T109|T131


In [69]:
# Grouping rows on SAB
print(f'Number of names before merging rows: {dutch_umls_snomed_drugs.shape[0]}')
dutch_umls_snomed_drugs = dutch_umls_snomed_drugs.groupby(['cui', 'name', 'type_ids'], as_index=False).agg({'ontologies' : '|'.join, 'name_status' : '|'.join}).copy()

# Clean name status column
dutch_umls_snomed_drugs.name_status = dutch_umls_snomed_drugs.name_status.apply(clean_name_status_column)
dutch_umls_snomed_drugs.sort_values(by=['cui', 'name_status'], ascending=[True, False], inplace=True)
dutch_umls_snomed_drugs.reset_index(drop=True,inplace=True)
print(f'Number of names after merging rows: {dutch_umls_snomed_drugs.shape[0]}')

Number of names before merging rows: 753449
Number of names after merging rows: 752536


In [70]:
# Check example
dutch_umls_snomed_drugs.loc[dutch_umls_snomed_drugs.cui == 'C0020740']

,cui,name,type_ids,ontologies,name_status
22234,C0020740,ibuprofen,T109|T121,ATC|RXNORM|DRUGBANK|SNOMEDCT_NL,P
22235,C0020740,(4-isobutylphenyl)-α-methylacetic acid,T109|T121,DRUGBANK,A
22236,C0020740,(RS)-ibuprofen,T109|T121,DRUGBANK,A
22237,C0020740,(±)-2-(p-isobutylphenyl)propionic acid,T109|T121,DRUGBANK,A
22238,C0020740,(±)-ibuprofen,T109|T121,DRUGBANK,A
22239,C0020740,(±)-p-isobutylhydratropic acid,T109|T121,DRUGBANK,A
22240,C0020740,(±)-α-methyl-4-(2-methylpropyl)benzeneacetic acid,T109|T121,DRUGBANK,A
22241,C0020740,2-(4-isobutylphenyl)propanoic acid,T109|T121,DRUGBANK,A
22242,C0020740,4-isobutylhydratropic acid,T109|T121,DRUGBANK,A
22243,C0020740,ibuprofene,T109|T121,DRUGBANK,A


### Save output

In [71]:
# Print statistics
print(f'Unique concepts: {len(dutch_umls_snomed_drugs.cui.unique())}')
print(f'Unique names: {len(dutch_umls_snomed_drugs.name.unique())}')
print(f'Ambiguous names: {len(dutch_umls_snomed_drugs) - len(dutch_umls_snomed_drugs.name.unique())}')
print(f'Total names in concepts: {len(dutch_umls_snomed_drugs)}')

# Save final concept table with drugs
dutch_umls_snomed_drugs.to_csv(output_file_with_drug_names, index=False)

Unique concepts: 366071
Unique names: 747600
Ambiguous names: 4936
Total names in concepts: 752536
